In [ ]:
'''
65. Valid Number
Hard

A valid number can be split up into these components (in order):

A decimal number or an integer.
(Optional) An 'e' or 'E', followed by an integer.
A decimal number can be split up into these components (in order):

(Optional) A sign character (either '+' or '-').
One of the following formats:
One or more digits, followed by a dot '.'.
One or more digits, followed by a dot '.', followed by one or more digits.
A dot '.', followed by one or more digits.
An integer can be split up into these components (in order):

(Optional) A sign character (either '+' or '-').
One or more digits.
For example, all the following are valid numbers: ["2", "0089", "-0.1", "+3.14", "4.", "-.9", "2e10", "-90E3", "3e+7", "+6e-1", "53.5e93", "-123.456e789"], while the following are not valid numbers: ["abc", "1a", "1e", "e3", "99e2.5", "--6", "-+3", "95a54e53"].

Given a string s, return true if s is a valid number.

 

Example 1:

Input: s = "0"
Output: true
Example 2:

Input: s = "e"
Output: false
Example 3:

Input: s = "."
Output: false
Example 4:

Input: s = ".1"
Output: true
 

Constraints:

1 <= s.length <= 20
s consists of only English letters (both uppercase and lowercase), digits (0-9), plus '+', minus '-', or dot '.'.
'''

In [34]:
class Solution:
    def isNumber(self, s: str) -> bool:
        s = s.lower()
        if s.isdigit():
            return True
        
        i = 0
        # check + - sign
        if s[0] == '+' or s[0] == '-':
            i += 1
            if len(s) == 1:
                return False # only + or -
        s = s[i:]
        if s[0]=='e': #e at front
            return False
        if s[0]=='.' and len(s)>1 and s[1]=='e':
            return False
        # check ., followed by one or more digits or E
        if '.' in s:
            # no two dot
            l = s.split('.')
            if len(l) > 2:
                return False
            if l[0]=='' and l[1]=='': # only dot, not digit
                return False
            if l[1]=='': # dot at the end
                return l[0].isdigit()
            if l[0]!='' and not l[0].isdigit():
                return False
            s = l[1] # len>0
        # check e or E, An 'e' or 'E', followed by an integer
        if 'e' in s:
            t = s.split('e')
            if len(t) > 2 or len(t[1])==0:
                return False
            if (t[0]=='' or t[0].isdigit()):
                s = t[1]
                if s[0] == '+' or s[0]=='-':
                    s = s[1:]
                return s.isdigit()
            else:
                return False
        return s.isdigit()
def test(s, expected):
    obj = Solution()
    result = obj.isNumber(s)
    print(s, result)
    assert result==expected

for b in ["2", "0089", "-0.1", "+3.14", "4.", "-.9", "2e10", "-90E3", "3e+7", "+6e-1", "53.5e93", "-123.456e789"]:
    test(b, True)
for b in [".e1", "abc", "1a", "1e", "e3", "99e2.5", "--6", "-+3", "95a54e53"]:
    test(b, False)
test("46.3e3", True)
test("46.e3", True)

2 True
0089 True
-0.1 True
+3.14 True
4. True
-.9 True
2e10 True
-90E3 True
3e+7 True
+6e-1 True
53.5e93 True
-123.456e789 True
.e1 False
abc False
1a False
1e False
e3 False
99e2.5 False
--6 False
-+3 False
95a54e53 False
46.3e3 True
46.e3 True


In [5]:
# solution 1: follow the rules
class Solution:
    def isNumber(self, s: str) -> bool:
        seen_digit = seen_exponent = seen_dot =  False
        for i, c in enumerate(s):
            if c.isdigit():
                seen_digit = True
            elif c in ["+", "-"]:
                if i > 0 and s[i - 1] != "e" and s[i - 1] != "E":
                    return False
            elif c in ["e", "E"]:
                if seen_exponent or not seen_digit:
                    return False
                seen_exponent = True
                seen_digit = False
            elif c == ".":
                if seen_dot or seen_exponent:
                    return False
                seen_dot = True
            else:
                return False
        
        return seen_digit

False

In [7]:
# solution 2 Deterministic Finite Automaton (DFA)
'''
Approach 2: Deterministic Finite Automaton (DFA)
Intuition

Let's now view Approach 1 from a different angle. There were 2^3 = 82 
3
 =8 states that the 3 boolean variables could be in. Each time we read a character in the string, we either stayed in the current state (boolean variables stayed the same) or we transitioned into a new state (boolean variables changed). What we've described above is a lot like a deterministic finite automaton. A DFA is a finite number of states, with transition rules to move between them.

Never heard of a DFA before?

DFA's are useful for solving many problems, including advanced dynamic programming problems such as 1411. Number of Ways to Paint N X 3 Grid. So if you're not yet familiar with them, we recommend that you read up on them. It will be worth it!

DFAs share a lot of similarities with the trie data structure. Recall that a trie is used to represent a dictionary of words, in a space-efficient manner. To check whether or not a word is in the dictionary, we would simultaneously traverse through the word and the trie. If we end at a node that is marked as a valid end-point, then we would return true. Otherwise, if we get "stuck", or end at a node that is not an end-point, we would return false. It's the same for a DFA: we start at a "root" node, and then check each character one by one, checking whether or not there is a valid transition we can make.

There are a few key differences between DFA's and tries, so keep these in mind while reading through the remainder of this section.

While a trie can only represent a finite number of strings (the given dictionary), a DFA can represent an infinite number of different strings.
While a trie can only move down the implicit tree, a DFA can essentially "loopback" to a higher level, or stay on the same level, or even the same node.
A trie is a type of tree, and a DFA is a type of directed graph.
Other than that, you can lean on your existing knowledge of tries to wrap your head around this new data structure.

Algorithm

The first step is to design our DFA. Picture the DFA as a directed graph, where each node is a state, and each edge is a transition labeled with a character group (digit, exponent, sign, or dot). There are two key steps to designing it.

Identify all valid combinations that the aforementioned boolean variables can be in. Each combination is a state. Draw a circle for each state, and label what it means.
For each state, consider what a character from each group would mean in the context of that state. Each group will either cause a transition into another state, or it will signify that the string is invalid. For each valid transition, draw a directed arrow between the two states and write the group next to the arrow.
Try this on your own before reading any further! Take a few minutes and try to design the DFA. Keep in mind that a state can point to itself. For example, with the input "12345", if we were to use the first approach, after the first character, none of the boolean variables change, so the state should not change. Therefore, the node should have an edge that points to itself, labeled by "digit". Another hint: the start node should have 3 outgoing edges labeled "digit", "sign", and "dot".






With our constructed DFA, our algorithm will be:

Initialize the DFA as an array of hash tables. Each hash table's keys will be a character group, and the values will be the state it should transition to. We can use the indexes of the array to handle state transitions. Set the currentState = 0.

Iterate through the input. For each character, first determine what group it belongs to. Then, check if that group exists in the current state's hash table. If it does, transition to the next state. Otherwise, return false.

At the end, check if we are currently in a valid end state: 1, 4, or 7.

If you're having trouble with your implementation, try to go through your DFA with a complicated case such as -123.456E+789. Follow along with your designed DFA, and if there is a bug, check which edge case went wrong and adjust the graph accordingly. Once your DFA is correctly designed, the coding part will be less challenging.

'''
class Solution(object):
    def isNumber(self, s):
        # This is the DFA we have designed above
        dfa = [
            {"digit": 1, "sign": 2, "dot": 3},
            {"digit": 1, "dot": 4, "exponent": 5},
            {"digit": 1, "dot": 3},
            {"digit": 4},
            {"digit": 4, "exponent": 5},
            {"sign": 6, "digit": 7},
            {"digit": 7},
            {"digit": 7}
        ]
        
        current_state = 0
        for c in s:
            if c.isdigit():
                group = "digit"
            elif c in ["+", "-"]:
                group = "sign"
            elif c in ["e", "E"]:
                group = "exponent"
            elif c == ".":
                group = "dot"
            else:
                return False

            if group not in dfa[current_state]:
                return False
            
            current_state = dfa[current_state][group]
        
        return current_state in [1, 4, 7]

(['12', '123'], ['', '123'], ['', '', '123'], ['', '1', '23'])

In [ ]:
'''
124. Binary Tree Maximum Path Sum
Hard

7054

449

Add to List

Share
A path in a binary tree is a sequence of nodes where each pair of adjacent nodes in the sequence has an edge connecting them. A node can only appear in the sequence at most once. Note that the path does not need to pass through the root.

The path sum of a path is the sum of the node's values in the path.

Given the root of a binary tree, return the maximum path sum of any path.

 

Example 1:


Input: root = [1,2,3]
Output: 6
Explanation: The optimal path is 2 -> 1 -> 3 with a path sum of 2 + 1 + 3 = 6.
Example 2:


Input: root = [-10,9,20,null,null,15,7]
Output: 42
Explanation: The optimal path is 15 -> 20 -> 7 with a path sum of 15 + 20 + 7 = 42.
 

Constraints:

The number of nodes in the tree is in the range [1, 3 * 104].
-1000 <= Node.val <= 1000
'''

In [ ]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def maxPathSum(self, root: Optional[TreeNode]) -> int:

In [ ]:
# solution
class Solution:
    def maxPathSum(self, root):
        """
        :type root: TreeNode
        :rtype: int
        """
        def max_gain(node):
            nonlocal max_sum
            if not node:
                return 0

            # max sum on the left and right sub-trees of node
            left_gain = max(max_gain(node.left), 0)
            right_gain = max(max_gain(node.right), 0)
            
            # the price to start a new path where `node` is a highest node
            price_newpath = node.val + left_gain + right_gain
            
            # update max_sum if it's better to start a new path
            max_sum = max(max_sum, price_newpath)
        
            # for recursion :
            # return the max gain if continue the same path
            return node.val + max(left_gain, right_gain)
   
        max_sum = float('-inf')
        max_gain(root)
        return max_sum

In [ ]:
'''
140. Word Break II
Hard

3782

452

Add to List

Share
Given a string s and a dictionary of strings wordDict, add spaces in s to construct a sentence where each word is a valid dictionary word. Return all such possible sentences in any order.

Note that the same word in the dictionary may be reused multiple times in the segmentation.

 

Example 1:

Input: s = "catsanddog", wordDict = ["cat","cats","and","sand","dog"]
Output: ["cats and dog","cat sand dog"]
Example 2:

Input: s = "pineapplepenapple", wordDict = ["apple","pen","applepen","pine","pineapple"]
Output: ["pine apple pen apple","pineapple pen apple","pine applepen apple"]
Explanation: Note that you are allowed to reuse a dictionary word.
Example 3:

Input: s = "catsandog", wordDict = ["cats","dog","sand","and","cat"]
Output: []
 

Constraints:

1 <= s.length <= 20
1 <= wordDict.length <= 1000
1 <= wordDict[i].length <= 10
s and wordDict[i] consist of only lowercase English letters.
All the strings of wordDict are unique.

'''

In [ ]:
class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> List[str]:
        

In [ ]:
# solution 1: Approach 1: Top-Down Dynamic Programming
class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> List[str]:
        wordSet = set(wordDict)
        # table to map a string to its corresponding words break
        # {string: [['word1', 'word2'...], ['word3', 'word4', ...]]}
        memo = defaultdict(list)

        #@lru_cache(maxsize=None)    # alternative memoization solution
        def _wordBreak_topdown(s):
            """ return list of word lists """
            if not s:
                return [[]]  # list of empty list

            if s in memo:
                # returned the cached solution directly.
                return memo[s]

            for endIndex in range(1, len(s)+1):
                word = s[:endIndex]
                if word in wordSet:
                    # move forwards to break the postfix into words
                    for subsentence in _wordBreak_topdown(s[endIndex:]):
                        memo[s].append([word] + subsentence)
            return memo[s]

        # break the input string into lists of words list
        _wordBreak_topdown(s)

        # chain up the lists of words into sentences.
        return [" ".join(words) for words in memo[s]]

In [ ]:
# solution 2: Bottom-Up Dynamic Programming
class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> List[str]:
        # quick check on the characters,
        #   otherwise it would exceed the time limit for certain test cases.
        if set(Counter(s).keys()) > set(Counter("".join(wordDict)).keys()):
            return []

        wordSet = set(wordDict)

        dp = [[]] * (len(s)+1)
        dp[0] = [""]

        for endIndex in range(1, len(s)+1):
            sublist = []
            # fill up the values in the dp array.
            for startIndex in range(0, endIndex):
                word = s[startIndex:endIndex]
                if word in wordSet:
                    for subsentence in dp[startIndex]:
                        sublist.append((subsentence + ' ' + word).strip())

            dp[endIndex] = sublist

        return dp[len(s)]